In [1]:
import os
from pathlib import Path

testfolder = str(Path().resolve().parent / 'TEMP' /  'sofia')

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in C:\Users\sayala\Documents\GitHub\bifacial_radiance\docs\TEMP\sofia


In [2]:
import bifacial_radiance as br
import numpy as np
import pandas as pd

In [3]:
# This information helps with debugging and getting support :)
import sys, platform
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)

Working on a  Windows 10
Python version  3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37) [MSC v.1916 64 bit (AMD64)]
Pandas version  2.1.0
bifacial_radiance version  0.4.2+161.g1692460.dirty


In [4]:
simulationname = 'tutorial_1'

# Location:
lat = 40.0583  # NJ
lon = -74.4057  # NJ

# Scene Parameters:
azimuth_ang=90 # Facing south
tilt = 30 # tilt.

# MakeModule Parameters
moduletype='test-module'
numpanels = 1  # AgriPV site has 3 modules along the y direction
module_x = 2 # m
module_y = 1 # m. slope we will measure
sensorsy=2
sensorsground=5

# SceneDict Parameters
pitch = 6 # m
albedo = 0.2  #'grass'     # ground albedo
clearance_height = 0.5 # m  
nMods = 4 # six modules per row.
nRows = 3  # 3 row


In [5]:
demo = br.RadianceObj(simulationname,path = testfolder)  
demo.setGround(albedo) 
epwfile = demo.getEPW(lat, lon) # NJ lat/lon 40.0583° N, 74.4057


path = C:\Users\sayala\Documents\GitHub\bifacial_radiance\docs\TEMP\sofia
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.
Getting weather file: USA_NJ_McGuire.AFB.724096_TMY3.epw
 ... OK!


In [6]:
#Valid options: mm_dd, mm_dd_HH, mm_dd_HHMM, YYYY-mm-dd_HHMM
metdata = demo.readWeatherFile(epwfile, coerce_year=2021, starttime='2021-06-01', endtime='2021-06-30_23')

8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos


Couple measurements:
* hub_height = 1.5 m
* pitch = 5.7 m
* modules are in 1-up portrait
* Field size is 10 rows x 20 modules
* Module are 2x1 m
* Pyranometers start 16in off east-way from the center of row 7, module 14
    
Suggestions:
* Do an hourly simulation. To calculate the tracker angle, look at the function and documentation of [`getSingleTimestampTrackerAngle`](https://bifacial-radiance.readthedocs.io/en/latest/generated/bifacial_radiance.RadianceObj.getSingleTimestampTrackerAngle.html#bifacial_radiance.RadianceObj.getSingleTimestampTrackerAngle)


In [7]:
# -- establish tracking angles
hub_height = 1.5 # m
pitch = 5.7 # m
sazm = 180  # Tracker axis azimuth
fixed_tilt_angle = None
gcr = 2 / pitch
cumulativesky = True

trackerParams = {'limit_angle':50,
             'angledelta':30, # 
             'backtrack':True,
             'gcr':gcr,
             'cumulativesky':cumulativesky,
             'azimuth': sazm,
             'fixed_tilt_angle': fixed_tilt_angle
             }

In [17]:
trackerdict = demo.set1axis(**trackerParams)

Saving file EPWs\1axis_-60.0.csv, # points: 90
Saving file EPWs\1axis_-30.0.csv, # points: 90
Saving file EPWs\1axis_-0.0.csv, # points: 90
Saving file EPWs\1axis_30.0.csv, # points: 90
Saving file EPWs\1axis_60.0.csv, # points: 90


In [18]:
if cumulativesky:
    demo.genCumSky1axis()
else:
    demo.gendaylit1axis()

message: There were 86 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_-60.0.rad
message: There were 90 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_-30.0.rad
message: There were 82 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_-0.0.rad
message: There were 89 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_30.0.rad
message: There were 88 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_60.0.rad


In [19]:
sceneDict = {'pitch':pitch, 
             'hub_height': hub_height,
             'nMods': 5,
             'nRows': 2,
             'tilt': fixed_tilt_angle,  
             'sazm': sazm
             }

In [20]:
trackerdict = demo.makeScene1axis(module=moduletype,sceneDict=sceneDict)
trackerdict = demo.makeOct1axis()


Making .rad files for cumulativesky 1-axis workflow
5 Radfiles created in /objects/

Making 5 octfiles in root directory.
Created 1axis_-60.0.oct
Created 1axis_-30.0.oct
Created 1axis_-0.0.oct
Created 1axis_30.0.oct
Created 1axis_60.0.oct


In [21]:
demo.trackerdict.keys()

dict_keys([-0.0, -30.0, -60.0, 30.0, 60.0])

In [23]:
trackerdict = demo.analysis1axis(customname = 'Module',
                                   sensorsy=2, modWanted=2,
                                   rowWanted=1) #sensorsground=2)

Linescan in process: 1axis_-60.0Module_Row1_Module2_Front
Linescan in process: 1axis_-60.0Module_Row1_Module2_Back
Saved: results\irr_1axis_-60.0Module_Row1_Module2.csv
FrontDict is:  {'Wm2': [46639.30000000001, 46712.12], 'x': [0.1015199, -0.0651468], 'y': [-2.01, -2.01], 'z': [1.366162, 1.654838], 'r': [46639.3, 46712.12], 'g': [46639.3, 46712.12], 'b': [46639.3, 46712.12], 'mattype': ['a1.0.a0.test-module.6457', 'a1.0.a0.test-module.6457'], 'title': '1axis_-60.0Module_Row1_Module2_Front'}
{'octfile': '1axis_-60.0.oct', 'name': '1axis_-60.0Module', 'hpc': False, 'modWanted': 2, 'rowWanted': 1, 'x': [0.1015199, -0.0651468], 'y': [-2.01, -2.01], 'z': [1.366162, 1.654838], 'rearZ': [1.355162, 1.643838], 'mattype': ['a1.0.a0.test-module.6457', 'a1.0.a0.test-module.6457'], 'rearMat': ['a1.0.a0.test-module.2310', 'a1.0.a0.test-module.2310'], 'Wm2Front': [46639.30000000001, 46712.12], 'Wm2Back': [5443.616, 5423.33], 'Back/FrontRatio': [0.11671735817824541, 0.11610112929789679], 'backRatio':

c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\main.py:2853: Warning: Index: -60.0. Problem with file. Error: 'AnalysisObj' object has no attribute 'keys'. Skipping
  warnings.warn('Index: {}. Problem with file. Error: {}. Skipping'.format(index,e), Warning)


In [ ]:
600

In [ ]:
## Loop for sensrosground = 1, 2, 3, 5, 10, 20, 30,50,100,150,200,250,300, 400, 500

